### rackcli layout - h4x VCV Rack layout management for patches

VCV Rack stores patches in JSON files, that's the good news. The not so good news is they're stored in the order they were added to the patch with ordinal cross references between modules and wires. This exercise is all about codifying some basic layout management services to facilitate patch construction and maintenance. As Rack matures, many of these capabilities will surely be added.

This exercise assumes your patch has plugin module width data, either from the `File.Save/SaveAs` in my 
[fork](https://github.com/dirkleas/Rack) or clicking `patch` button on 
[DLwigglz r4xH4x](https://github.com/dirkleas/DLwigglz) and using `patch.json` instead of `_layout.vcv` -- you don't need a 
full `catalog.json` for this since we're not dynamically adding plugin modules to the patch, only reorgaizing them. Don't miss the companion [video](https://www.youtube.com/watch?v=ZTS-T8-mn1c)!

In [1]:
from typing import *
from pprint import pprint as pp
import json, re, copy

Lets start our layout experiment with left and right alignment.

In [2]:
rackdir = '/Users/dirkleas/Desktop/h4x/_queue_/vcv/Rack'

catalog = json.load(open(rackdir + '/catalog.json'))
patch = json.load(open(rackdir + '/_layout.vcv'))

def hp(pixels: int) -> int:
    'convert pixels to hp'
    return(pixels / 15)

def layout(patch: Dict) -> Dict[int, List]:
    'generate ordered layout model for patch with column, ordinal, width'
    layout = {}
    for i,m in enumerate(patch['modules']): layout.setdefault(m['pos'][1], []).append([m['pos'][0], i, m['width']])
    return({k:sorted(v, key=lambda x: x[0]) for (k,v) in layout.items()})

pl = layout(patch)
print(pl, catalog['width'])

def align_left(layout: Dict[int, List], rows: List, start: int=0) -> Dict[int, List]:
    'left-align row modules on specified rows with optional 0-offset starting module, use slicing once available'
    for r in rows:
        sum = 0 if start == 0 else layout[r][start-1][2] + (15*layout[r][start-1][0])
        for m in layout[r][start:]:
            m[0] = int(hp(sum))
            sum += m[2]
    return(layout)

def align_right(layout: Dict[int, List], rows: List, max_width, start: int=0) -> Dict[int, List]:
    'left-align row modules on specified rows with optional 0-offset starting module, use slicing once available'
    for r in rows:
        sum = max_width - 15 # allow 1HP for scrollbar
        for m in layout[r][:None if start==0 else start-1:-1]:
            m[0] = int(hp(sum)) - int(hp(m[2]))
            sum -= m[2]
    return(layout)

pl = align_left(pl, [0,])
pl = align_right(pl, [1,], catalog['width'])
# pl = align_left(pl, [0,], 1) # shift from 1th item on
# pl = align_right(pl, [1,], catalog['width'], 1)
print(pl, catalog['width'])

def apply_layout(layout: Dict, patch: Dict) -> Dict:
    for r in layout.values():
        for c in r: patch['modules'][c[1]]['pos'][0] = c[0]
    return(patch)

json.dump(apply_layout(pl, patch), open(rackdir + '/_layout2.vcv', 'w'), indent=2, ensure_ascii=False)

{0: [[3, 1, 270], [29, 2, 90], [85, 0, 150]], 1: [[34, 3, 150], [54, 5, 150], [66, 4, 120], [78, 6, 90]]} 1440
{0: [[0, 1, 270], [18, 2, 90], [24, 0, 150]], 1: [[61, 3, 150], [71, 5, 150], [81, 4, 120], [89, 6, 90]]} 1440


Slicing allows us to specify a subset of modules in a patch by row. Our simplified slicing syntax is as follows: `ROW[START_COLUMN:END_COLUMN]` where `ROW` is required, and `START_COLUMN` and `END_COLUMN` are optional. The square brackets and colon are always required. Slices are typically specified in a list, and always passed as individual strings. Here is an example of a list of slices:

`['0[1:5]', '0[:]', '0[:3]', '0[5:]', '0[:-3]', '0[-4:-2]']`

Here's a convenience function for coverting a list of slice strings to a list of three individual numbers. While slicing can be applied to any 2D list, we'll be slicing modules on the rows in our patches.

In [3]:
def slicer(slices: List[str]) -> List[List]:
    'covert list of slice strings to list slice-ready lists, base 0'
    ss = []
    for slice in slices:
        ss.append(list(map(lambda d: int(d) if d.lstrip('-+').isdigit() else None, 
                           re.match(r'(\d+)\[(-?\d*):(-?\d*)]', slice).groups())))
    return(ss)

x = [list(range(10)),[]] # 2d rack grid
slices = ['0[1:5]', '0[:]', '0[:3]', '0[5:]', '0[:-3]', '0[-4:-2]']
print('rack:', x)
print('slices:', slices)
slices = slicer(slices)
print('slices:', slices)
print('rack slices:\n ', list(map(lambda s: x[s[0]][s[1]:s[2]], slices)))

rack: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], []]
slices: ['0[1:5]', '0[:]', '0[:3]', '0[5:]', '0[:-3]', '0[-4:-2]']
slices: [[0, 1, 5], [0, None, None], [0, None, 3], [0, 5, None], [0, None, -3], [0, -4, -2]]
rack slices:
  [[1, 2, 3, 4], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2], [5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6], [6, 7]]


Now that we can slice, we can implement import and export to facilitate the resuse of sub-patches. It's important to consider the ordinal values/references used for patch "modules" and "wires" references/cross-references. "New" patch fragments are subject to the following:

1. same version
1. ordinal extension (e.g. new modules appended to existing modules, and wires input|outputModuleId increased by ordinal offset)
1. appropriately laid out on the rack grid.

Here's an implementation of import and export, with slicing support on export. It's quite possible that an import will cause one or more modules to "collide" in your patch where one sits on top of others. If this happens, use one of the alignment functions above to fix the layout.

In [4]:
p1 = json.load(open(rackdir + '/imp1.vcv'))
p2 = json.load(open(rackdir + '/imp2.vcv'))
l1 = json.load(open(rackdir + '/_layout.vcv'))

def import_patch(p1: Dict, p2: Dict) -> Dict:
    'import p2 into p1, if module collisions occur, use align_*()'
    if not p1['version'] == p2['version']: return(None)
    ordinal_offset = len(p1['modules'])
    for w in p2['wires']: w['inputModuleId'] += ordinal_offset; w['outputModuleId'] += ordinal_offset
    p1['modules'] += p2['modules']; p1['wires'] += p2['wires']
    return(p1)

def export_patch(p: Dict, slices: List[Tuple[str]], filename: str) -> Dict:
    'export patch slices where slice is (row, start_col, stop_col)'
    p2 = copy.deepcopy(p); p2['modules'] = []; p2['wires'] = []
    l = layout(p); ordinals = []
    for s in slicer(slices): ordinals += [o[1] for o in l[s[0]][s[1]:s[2]]] # slice layout, keep ordinal
    for o in ordinals: p2['modules'].append(p['modules'][o])
    for w in p['wires']: # wires -- add in|outputModuleId both in ordinals
        if w['inputModuleId'] in ordinals and w['outputModuleId'] in ordinals:
            w['inputModuleId'] = ordinals.index(w['inputModuleId']) # adjust references
            w['outputModuleId'] = ordinals.index(w['outputModuleId'])
            p2['wires'].append(w)
    json.dump(p2, open(filename, 'w'), indent=2, ensure_ascii=False)
    return(p2)

p3 = import_patch(p1, p2)
json.dump(p3, open(rackdir + '/imp3.vcv', 'w'), indent=2, ensure_ascii=False)

export_patch(l1, ['0[1:2]', '1[1:-1]'], rackdir + '/_layout_exp.vcv')

{'version': '0.6.1',
 'lastMousePos': [60, 0],
 'width': 1440,
 'height': 878,
 'modules': [{'plugin': 'DLwigglz',
   'version': '0.6.0',
   'model': 'DWLwigglz-r4xH4x',
   'pos': [29, 0],
   'width': 90,
   'params': [{'paramId': 0, 'value': 0.0}, {'paramId': 1, 'value': 0.0}]},
  {'plugin': 'Fundamental',
   'version': '0.6.0',
   'model': 'LFO',
   'pos': [54, 1],
   'width': 150,
   'params': [{'paramId': 0, 'value': 1.0},
    {'paramId': 1, 'value': 1.0},
    {'paramId': 2, 'value': -1.0},
    {'paramId': 3, 'value': 0.0},
    {'paramId': 5, 'value': 0.5},
    {'paramId': 4, 'value': 0.0},
    {'paramId': 6, 'value': 0.0}]},
  {'plugin': 'Fundamental',
   'version': '0.6.0',
   'model': 'VCF',
   'pos': [66, 1],
   'width': 120,
   'params': [{'paramId': 0, 'value': 0.5},
    {'paramId': 1, 'value': 0.5},
    {'paramId': 2, 'value': 0.0},
    {'paramId': 3, 'value': 0.0},
    {'paramId': 4, 'value': 0.0}]}],
 'wires': [{'color': {'r': 0.788235366,
    'g': 0.717647076,
    'b': 0.